IMPORTS

In [23]:
import random
import numpy as np
import pickle
import os
import copy

LOAD DATA

In [24]:
#Load Train Data
x = []
y = []
filename = 'cifar-10-batches-py/data_batch_%d'
for b in range(1,6):
    f = os.path.join(filename  % (b, ))
    with open(f, 'r') as f:
        datadict = pickle.load(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
    x.append(X)
    y.append(Y)    
    
x_train = np.concatenate(x)
y_train = np.concatenate(y)

#Load Test Data
f = 'cifar-10-batches-py/test_batch'
with open(f, 'r') as f:
    datadict = pickle.load(f)
    x_test = datadict['data']
    y_test = datadict['labels']
    x_test = x_test.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
    y_test = np.array(y_test)

Get RAW Data

In [25]:
num_training = 49000
num_val = 1000
num_test = 10000

# subsample the data for validation set
mask = xrange(num_training, num_training + num_val)
x_val_raw = x_train[mask]
y_val_raw = y_train[mask]

mask = xrange(num_training)
x_train_raw = x_train[mask]
y_train_raw = y_train[mask]

mask = xrange(num_test)
x_test_raw = x_test[mask]
y_test_raw = y_test[mask]

Preprocess raw data

In [26]:
# Preprocessing: reshape the image data into rows
x_train = np.reshape(x_train_raw, (x_train_raw.shape[0], -1)) # [49000, 3072]
x_val = np.reshape(x_val_raw, (x_val_raw.shape[0], -1)) # [1000, 3072]
x_test = np.reshape(x_test_raw, (x_test_raw.shape[0], -1)) # [10000, 3072]
    
# Normalize the data: subtract the mean image
mean_image = np.mean(x_train, axis = 0)
x_train -= mean_image
x_val -= mean_image
x_test -= mean_image
    
# Add bias dimension and transform into columns
x_train = np.hstack([x_train, np.ones((x_train.shape[0], 1))]).T
x_val = np.hstack([x_val, np.ones((x_val.shape[0], 1))]).T
x_test = np.hstack([x_test, np.ones((x_test.shape[0], 1))]).T

y_train = y_train_raw
y_val = y_val_raw
y_test = y_test_raw

Set up and train 10 logistic classifier

In [27]:
# train 10 logistic classifier
from linear_classifier import Logistic

logistic_classifiers = []
num_classes = np.max(y_train) + 1
losses = []
for i in xrange(num_classes):
    print '\nThe %d/%dth logistic classifier training...' % (i+1, num_classes)
    y_train_logistic = copy.deepcopy(y_train)
    idxs_i = y_train_logistic == i
    y_train_logistic[idxs_i] = 1
    y_train_logistic[-idxs_i] = 0
    logistic = Logistic()
    loss = logistic.train(x_train, y_train_logistic, method='sgd', batch_size=200, learning_rate=1e-6,
              reg = 1e3, num_iters=1000, verbose=True, vectorized=True)
    losses.append(loss)
    logistic_classifiers.append(logistic)


The 1/10th logistic classifier training...
iteration 0/1000: loss 3.066540


/Users/ivanfilho/.local/lib/python2.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: numpy boolean negative (the unary `-` operator) is deprecated, use the bitwise_xor (the `^` operator) or the logical_xor function instead.
  if sys.path[0] == '':


iteration 100/1000: loss 2.001810
iteration 200/1000: loss 1.705051
iteration 300/1000: loss 1.500079
iteration 400/1000: loss 1.364895
iteration 500/1000: loss 1.188604
iteration 600/1000: loss 1.144279
iteration 700/1000: loss 1.008724
iteration 800/1000: loss 0.940295
iteration 900/1000: loss 0.919573

The 2/10th logistic classifier training...
iteration 0/1000: loss 3.777222
iteration 100/1000: loss 1.997938
iteration 200/1000: loss 1.734254
iteration 300/1000: loss 1.471140
iteration 400/1000: loss 1.420673
iteration 500/1000: loss 1.207401
iteration 600/1000: loss 1.154891
iteration 700/1000: loss 1.016298
iteration 800/1000: loss 1.004904
iteration 900/1000: loss 0.891820

The 3/10th logistic classifier training...
iteration 0/1000: loss 2.909394
iteration 100/1000: loss 2.193496
iteration 200/1000: loss 1.810792
iteration 300/1000: loss 1.557878
iteration 400/1000: loss 1.373106
iteration 500/1000: loss 1.216788
iteration 600/1000: loss 1.142319
iteration 700/1000: loss 1.05747

Prediction by using trained 10 logistic classifiers

In [28]:
# Compute the accuracy of training data and validation data
def predict_one_vs_all(logistic_classifiers, X, num_classes):
    scores = np.zeros((num_classes, X.shape[1]))
    for i in xrange(num_classes):
        logistic = logistic_classifiers[i]
        scores[i, :] = logistic.predict(X)[1]
    pred_X = np.argmax(scores, axis=0)
    return pred_X

pred_train_one_vs_all = predict_one_vs_all(logistic_classifiers, x_train, num_classes)
pred_val_one_vs_all = predict_one_vs_all(logistic_classifiers, x_val, num_classes)
pred_test_one_vs_all = predict_one_vs_all(logistic_classifiers, x_test, num_classes)
print 'Training dataset accuracy: %f' % (np.mean(y_train == pred_train_one_vs_all))
print 'Validation dataset accuracy: %f' % (np.mean(y_val == pred_val_one_vs_all))
print 'Test datast accuracy: %f' % (np.mean(y_test == pred_test_one_vs_all))

Training dataset accuracy: 0.347367
Validation dataset accuracy: 0.340000
Test datast accuracy: 0.339100
